In [1]:
import requests
import json
import pandas
import datetime
from pandas import HDFStore

In [2]:
Exchange_Info_API = 'https://api.binance.com/api/v3/exchangeInfo'
Price_Info_API = 'https://api.binance.com/api/v3/klines'

In [3]:
def symbols_info():
    data = json.loads(requests.get(Exchange_Info_API).text)
    cols = ['symbol','status','baseAsset','quoteAsset']
    data = [[i[j] for j in cols] for  i in data['symbols']]
    return pandas.DataFrame(data,columns=cols) 

In [59]:
def Price_info(symbol,step='1m',start=None,end=None,limit=500, base = '',quote='') :

    par = {
        'symbol':symbol,
        'startTime':start,
        'endTime':end,
        'interval':step,
        'limit':limit}


    cols =[
        'Kline open time',   #if a range of time
        'Open price',       #Price ar the time open
        'High price',       #max in time
        'Low price',        #Min in time
        'Close price',      #Price at the time close
        'Volume',           #Sum in time
        'Kline Close time',
        'Quote asset volume',   #Sum in time
        'Number of trades',     #Sum in time
        'Taker buy base asset volume',  #Sum in time
        'Taker buy quote asset volume', #Sum in time
        'Unused field, ignore'] # not use 
    p=pandas.DataFrame(json.loads(requests.get(Price_Info_API,params=par).text),columns=cols)
    p.insert(0,'quote', quote)
    p.insert(0,'base', base)
    p.insert(0,'Symbol', symbol)
    return p

In [42]:
def Date_to_timestamp(_time,_format = '%d/%m/%Y %H:%M:%S'):
    return int(datetime.datetime.timestamp(datetime.datetime.strptime(_time,_format ))*1000)

-------------------

In [57]:
symbols = symbols_info()

In [60]:
data = pandas.concat(
    [Price_info(
        i['symbol'],
        '30m',
        Date_to_timestamp('21/10/2022 00:00:00'),
        Date_to_timestamp('10/11/2022 00:00:00'),
        1000,
        i['baseAsset'],
        i['quoteAsset']
        ) for _,i in symbols[symbols['status']=='TRADING'].iterrows()
    ],
    ignore_index=True)

In [61]:
data

,Symbol,base,quote,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,"Unused field, ignore"
0,ETHBTC,ETH,BTC,1666285200000,0.06727900,0.06728400,0.06713700,0.06726100,1678.27060000,1666286999999,112.81479874,3360,599.98040000,40.32979955,0
1,ETHBTC,ETH,BTC,1666287000000,0.06726100,0.06732000,0.06724600,0.06727800,1671.44470000,1666288799999,112.47082167,3039,640.25320000,43.08404508,0
2,ETHBTC,ETH,BTC,1666288800000,0.06727800,0.06743800,0.06718200,0.06743700,2441.54230000,1666290599999,164.26715006,4622,1094.13180000,73.62494828,0
3,ETHBTC,ETH,BTC,1666290600000,0.06743700,0.06761200,0.06743700,0.06761200,1789.04260000,1666292399999,120.85124398,4405,988.24110000,66.75865608,0
4,ETHBTC,ETH,BTC,1666292400000,0.06761100,0.06762100,0.06745200,0.06751100,1212.35940000,1666294199999,81.86182297,2816,520.46620000,35.14032281,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387693,HFTUSDT,HFT,USDT,1668006000000,0.62480000,0.63790000,0.61110000,0.61330000,931642.00000000,1668007799999,583553.93953000,2295,414927.30000000,260149.78643000,0
1387694,HFTUSDT,HFT,USDT,1668007800000,0.61450000,0.62000000,0.56430000,0.57170000,2464439.60000000,1668009599999,1434603.32395000,4564,1219866.30000000,710327.10684000,0
1387695,HFTUSDT,HFT,USDT,1668009600000,0.57100000,0.58900000,0.54700000,0.55560000,1464321.10000000,1668011399999,832308.56927000,3407,624211.00000000,354941.04038000,0
1387696,HFTUSDT,HFT,USDT,1668011400000,0.55450000,0.59800000,0.55200000,0.57330000,1353333.60000000,1668013199999,783057.56581000,3379,574310.90000000,332213.45400000,0


In [62]:
hdf= pandas.HDFStore('Saved_Data','w')
hdf.put('data',data)
hdf.close()

In [63]:
a = pandas.HDFStore('Saved_Data','r')
df = a.get('data')
a.close()
df

,Symbol,base,quote,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,"Unused field, ignore"
0,ETHBTC,ETH,BTC,1666285200000,0.06727900,0.06728400,0.06713700,0.06726100,1678.27060000,1666286999999,112.81479874,3360,599.98040000,40.32979955,0
1,ETHBTC,ETH,BTC,1666287000000,0.06726100,0.06732000,0.06724600,0.06727800,1671.44470000,1666288799999,112.47082167,3039,640.25320000,43.08404508,0
2,ETHBTC,ETH,BTC,1666288800000,0.06727800,0.06743800,0.06718200,0.06743700,2441.54230000,1666290599999,164.26715006,4622,1094.13180000,73.62494828,0
3,ETHBTC,ETH,BTC,1666290600000,0.06743700,0.06761200,0.06743700,0.06761200,1789.04260000,1666292399999,120.85124398,4405,988.24110000,66.75865608,0
4,ETHBTC,ETH,BTC,1666292400000,0.06761100,0.06762100,0.06745200,0.06751100,1212.35940000,1666294199999,81.86182297,2816,520.46620000,35.14032281,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387693,HFTUSDT,HFT,USDT,1668006000000,0.62480000,0.63790000,0.61110000,0.61330000,931642.00000000,1668007799999,583553.93953000,2295,414927.30000000,260149.78643000,0
1387694,HFTUSDT,HFT,USDT,1668007800000,0.61450000,0.62000000,0.56430000,0.57170000,2464439.60000000,1668009599999,1434603.32395000,4564,1219866.30000000,710327.10684000,0
1387695,HFTUSDT,HFT,USDT,1668009600000,0.57100000,0.58900000,0.54700000,0.55560000,1464321.10000000,1668011399999,832308.56927000,3407,624211.00000000,354941.04038000,0
1387696,HFTUSDT,HFT,USDT,1668011400000,0.55450000,0.59800000,0.55200000,0.57330000,1353333.60000000,1668013199999,783057.56581000,3379,574310.90000000,332213.45400000,0
